In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 16008, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 16008 (delta 109), reused 139 (delta 92), pack-reused 15831
Receiving objects: 100% (16008/16008), 14.60 MiB | 15.95 MiB/s, done.
Resolving deltas: 100% (10983/10983), done.


In [ ]:
# YOLO 모델 로드
%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.4/612.4 kB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00


In [ ]:
%cat /content/drive/MyDrive/pipe/images/data.yaml

names:
- crack
- corrosion
- boring
nc: 3
train: /content/drive/MyDrive/pipe/train.txt
val: /content/drive/MyDrive/pipe/val.txt


In [ ]:
# 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 변환하는 glob 사용
%cd /
from glob import glob

img_list = glob('/content/drive/MyDrive/pipe/images/*.jpg')

print(len(img_list))

/
758


In [ ]:
# 검증데이터와 학습데이터 분할
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))

606 152


In [ ]:
# 파일 이름 수정
with open('/content/drive/MyDrive/pipe/train.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')
with open('/content/drive/MyDrive/pipe/val.txt', 'w') as f:
    f.write('\n'.join(val_img_list) + '\n')

In [ ]:
import yaml

In [ ]:
# yaml파일을 파싱해서 python 객체로 불러오기
with open('/content/drive/MyDrive/pipe/images/data.yaml', 'r')  as f:
    data = yaml.full_load(f)

print(data)

data['train'] = '/content/drive/MyDrive/pipe/train.txt'
data['val'] = '/content/drive/MyDrive/pipe/val.txt'

with open('/content/drive/MyDrive/pipe/images/data.yaml', 'w') as f:
    yaml.dump(data, f)

print(data)

{'names': ['crack', 'corrosion', 'boring'], 'nc': 3, 'train': '/content/drive/MyDrive/pipe/train.txt', 'val': '/content/drive/MyDrive/pipe/val.txt'}
{'names': ['crack', 'corrosion', 'boring'], 'nc': 3, 'train': '/content/drive/MyDrive/pipe/train.txt', 'val': '/content/drive/MyDrive/pipe/val.txt'}


In [ ]:
# 모델 학습 후 가중치와 결과 저장하기
%cd /content/yolov5/

!python train.py --img 720 --batch 16 --epochs 100 --data /content/drive/MyDrive/pipe/images/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name gun_yolov5s_results

/content/yolov5
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/drive/MyDrive/pipe/images/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=720, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=gun_yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-187-g0004c74 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/gun_yolov5s_results10/weights/best.pt --source "/content/drive/MyDrive/videos/보유배관_200mm_9.mp4" --conf 0.6

detect: weights=['/content/yolov5/runs/train/gun_yolov5s_results10/weights/best.pt'], source=/content/drive/MyDrive/videos/보유배관_200mm_9.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-187-g0004c74 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/1613) /content/drive/MyDrive/videos/보유배관_200mm_9.mp4: 384x640 1 corrosion, 86.5ms
video 1/1 (2/1613) /content/drive/MyDrive/videos/보유배관_200mm_9.mp4: 384x640 (no detections), 6.5ms
video 1/1 (3/1613) /content/drive/MyDrive/videos/보유배관_200mm_9.mp4: 384x640 (no

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/gun_yolov5s_results10/weights/best.pt --source "/content/drive/MyDrive/videos/보유배관_200mm_8.mp4" --conf 0.6

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
video 1/1 (106/5103) /content/drive/MyDrive/videos/보유배관_200mm_8.mp4: 384x640 4 corrosions, 6.6ms
video 1/1 (107/5103) /content/drive/MyDrive/videos/보유배관_200mm_8.mp4: 384x640 2 corrosions, 6.6ms
video 1/1 (108/5103) /content/drive/MyDrive/videos/보유배관_200mm_8.mp4: 384x640 4 corrosions, 7.8ms
video 1/1 (109/5103) /content/drive/MyDrive/videos/보유배관_200mm_8.mp4: 384x640 3 corrosions, 6.7ms
video 1/1 (110/5103) /content/drive/MyDrive/videos/보유배관_200mm_8.mp4: 384x640 2 corrosions, 6.4ms
video 1/1 (111/5103) /content/drive/MyDrive/videos/보유배관_200mm_8.mp4: 384x640 3 corrosions, 6.7ms
video 1/1 (112/5103) /content/drive/MyDrive/videos/보유배관_200mm_8.mp4: 384x640 3 corrosions, 6.7ms
video 1/1 (113/5103) /content/drive/MyDrive/videos/보유배관_200mm_8.mp4: 384x640 1 corrosion, 6.5ms
video 1/1 (114/5103) /content/drive/MyDrive/videos/보유배관_200mm_8.mp4: 384x640 4 corrosions, 6.7ms
video 1/1 (115/5103) /content/drive/MyDrive/videos/보유배관_200mm_8.mp4: 384x640 (no detections)

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/gun_yolov5s_results10/weights/best.pt --source "/content/drive/MyDrive/videos/보유배관_200mm_7.mp4" --conf 0.6

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
video 1/1 (3254/8251) /content/drive/MyDrive/videos/보유배관_200mm_7.mp4: 384x640 1 corrosion, 6.5ms
video 1/1 (3255/8251) /content/drive/MyDrive/videos/보유배관_200mm_7.mp4: 384x640 1 corrosion, 6.4ms
video 1/1 (3256/8251) /content/drive/MyDrive/videos/보유배관_200mm_7.mp4: 384x640 1 corrosion, 6.6ms
video 1/1 (3257/8251) /content/drive/MyDrive/videos/보유배관_200mm_7.mp4: 384x640 2 corrosions, 6.3ms
video 1/1 (3258/8251) /content/drive/MyDrive/videos/보유배관_200mm_7.mp4: 384x640 1 corrosion, 6.4ms
video 1/1 (3259/8251) /content/drive/MyDrive/videos/보유배관_200mm_7.mp4: 384x640 2 corrosions, 6.4ms
video 1/1 (3260/8251) /content/drive/MyDrive/videos/보유배관_200mm_7.mp4: 384x640 2 corrosions, 6.6ms
video 1/1 (3261/8251) /content/drive/MyDrive/videos/보유배관_200mm_7.mp4: 384x640 2 corrosions, 7.3ms
video 1/1 (3262/8251) /content/drive/MyDrive/videos/보유배관_200mm_7.mp4: 384x640 2 corrosions, 6.6ms
video 1/1 (3263/8251) /content/drive/MyDrive/videos/보유배관_200mm_7.mp4: 384x640 2 corros